## import important libraries

In [16]:
import cv2
import math
import os
import librosa
import soundfile as sf
import subprocess
import platform

## get the frame rate,width,height of the video

In [17]:
def getFps(video_read):
  fps = video_read.get(cv2.CAP_PROP_FPS)
  frame_width = video_read.get(cv2.CAP_PROP_FRAME_WIDTH)
  frame_height = video_read.get(cv2.CAP_PROP_FRAME_HEIGHT)
  total_Frame_Number = int(video_read.get(cv2.CAP_PROP_FRAME_COUNT))
  return fps, frame_width, frame_height, total_Frame_Number


## Get the exact frame number of a video with respect to given time

In [18]:
def getFrameNumber(fps, milisecond):
  frame_number = math.floor((fps/1000)*milisecond)
  return frame_number  

  

## Convert the time stamp into milisecond

In [19]:
def minutes2mili(tm):
  tm_split = tm.split('|')
  tm_mili =(int(tm_split[0])*60000)+(int(tm_split[1])*1000)+int(tm_split[2])
  return tm_mili

## Function gives all the frames during a perticular given time duration

In [20]:
def getFrame(video_path,first_time_instant, second_time_instant,path_file):
  basename = os.path.basename(video_path)
  basename = basename.split('.')
  file_name = basename[0]+'_'+'trimmed'+'.'+basename[1]
  print(file_name)
  dirname = path_file
  count =0
  frame_list = []
  first_time_mili = minutes2mili(first_time_instant)
  second_time_mili = minutes2mili(second_time_instant)
  video_cap = cv2.VideoCapture(video_path)

  fps, w, h, f_num = getFps(video_cap)
  total_length_video = (f_num/fps)*1000
  if(first_time_mili < 0):
    print("Sorry time stamp is invalid!!")
    return -1
  elif (second_time_mili > total_length_video):
    print("Sorry time stamp exceeded from the actual video's time stamp !!!")
    print("total timestamp of your video in sec is : ", total_length_video/1000)
    return -1
  else:
      starting_frame = getFrameNumber(fps,first_time_mili)
      ending_frame = getFrameNumber(fps, second_time_mili)
      while(video_cap.isOpened()):
        success, frame = video_cap.read()
        count += 1

        if count >= starting_frame and count <= ending_frame:
          frame_list.append(frame)
          if count == ending_frame:
            break


  videoFile = createVideo(frame_list, fps,int(w),int(h), file_name,dirname)
  return videoFile


## Function returns all the cropped frame

In [21]:
def getCropFrame(video_path,a,b,c,d,path_file):
  basename = os.path.basename(video_path)
  basename = basename.split('.')
  file_name = basename[0]+'_'+'cropped'+'.'+basename[1]
  dirname = path_file
  video_cap = cv2.VideoCapture(video_path)
  
  list_frame = []
  count =0
  fps, w, h, f_num = getFps(video_cap)
  
  if (a<0 or b<0):
    print("invalid x1 or y1 coordinate!!")
    print("please put the write coordinate")
    return -1
  elif(c>w):
    print("width of the frame is exceeded!!")
    print("the weidth frame limit will be maximum: ",w)
    return -1

  elif(d>h):
    print("Height of the frame is exceeded!!!")
    print("the height of the frame limit will be maximum: ",h)
    return -1

  elif(a>c):
    print("Sorry invalid coordinate!!")
    return -1

  elif(b>d):
    print("Sorry invalid coordinate!!")
    return -1
  else:
    while(video_cap.isOpened()):
      success, frame = video_cap.read()
      count += 1
      cropped_frame = frame[int(b):int(d) , int(a):int(c)]
      list_frame.append(cropped_frame)
    
      if f_num == count:
        break
  height = list_frame[0].shape[0]
  weidth = list_frame[0].shape[1]
  videoFile = createVideo(list_frame, fps,weidth, height,file_name, dirname)
  return videoFile


## Function for create a video for given 
#1.frames
#2.fps
#3.weigth
#4.height 
#5.basename of a file and dirname of a file

In [22]:
def createVideo(list_of_frame, fps, w, h,basename_file,folder_des):
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    saved_file = folder_des+'/'+basename_file
    writer = cv2.VideoWriter(saved_file, fourcc, fps, (w, h))
    
    for i in range (len(list_of_frame)):
      writer.write(list_of_frame[i])
  
    writer.release()
    return saved_file





## Function for resizing the frame

In [23]:
def getFrameResize(video_path,weight, height,path_file):
  basename = os.path.basename(video_path)
  basename = basename.split('.')
  file_name = basename[0]+'_'+'resized'+'.'+basename[1]
  dirname = path_file
  video_cap = cv2.VideoCapture(video_path)
  list_frame = []
  count= 0
  fps, w, h, f_num = getFps(video_cap)

  if(weight <=0 or height <=0):

    print('weidth or height are invalid!!!')
    return -1
  else:
    while(video_cap.isOpened()):
      success, frame = video_cap.read()
      count += 1
  
      res_frame = cv2.resize(frame, (weight, height), interpolation = cv2.INTER_LINEAR)
      list_frame.append(res_frame)
   
      
      if count == f_num:
        break
  videofile = createVideo(list_frame, fps, weight, height,file_name,dirname)
  return videofile  




## Function for resizing the frame maintainig the aspect ration

In [24]:
def getFrameAspect(video_path, weight,path_file):
  basename = os.path.basename(video_path)
  basename = basename.split('.')
  file_name = basename[0]+'_'+'resized'+'.'+basename[1]
  dirname = path_file
  video_cap = cv2.VideoCapture(video_path)
  fps, w, h, f_num = getFps(video_cap)
  height = weight
  lst_fr = []
  count = 0
  if weight == 0:
    print("invalid weidth!!!")
    return -1
  else:

    while (video_cap.isOpened()):
      success, frame = video_cap.read()
      count += 1
      res_frame  =cv2.resize(frame, (weight, height))
      lst_fr.append(res_frame)

      if count == f_num:
        break
  video_file = createVideo(lst_fr, weight, height, file_name,dirname)
  return video_file

## Get the sample number of audio

In [25]:
def getAudioSampleNum(time, sf):
  milisecond = minutes2mili(time)
  sample_num = math.floor((sf/1000)*milisecond)
  return sample_num


## Function for trim the audio given time duration

In [26]:
def trimAudio(video_path, starting_time, ending_time,dirname):
  basename = os.path.basename(video_path)
  basename = basename.split('.')
  file_name = basename[0]+'_'+'trimmed'+'.'+'wav'
  path_name = dirname
  audio, sr = librosa.load(video_path, 16000)
  sample_start = getAudioSampleNum(starting_time,sr)
  sample_end = getAudioSampleNum(ending_time,sr)
  trimmed_audio = audio[sample_start: sample_end]
  audioFile = writeAudio(file_name, trimmed_audio, sr, path_name)
  return audioFile

## Function for write the audio

In [27]:
def writeAudio(basename,audio, sr,folder_des):
  saved_file = folder_des+'/'+basename
  sf.write(saved_file,audio, sr)
  return saved_file

## merge audio with video

In [28]:
def audioMergevideo(video, audio, saved_file):
  basename = os.path.basename(video)
  basename = basename.split('.')
  final_video = saved_file + '/'+basename[0]+'Merged'+'.'+basename[1]
  command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 {}'.format(audio, video, final_video)
  subprocess.call(command, shell=platform.system() != 'Windows')



## Function for extract audio from the video

In [39]:
def getAudio(video_path, path_audio):
  video_path_base = os.path.basename(video_path)
  basename = video_path_base.split('.')
  path_audio = basename[0]+'.'+'wav'
  print(path_audio)
  
  subprocess.call(['ffmpeg','-i' ,video_path, path_audio])
  return path_audio


## User main code


In [ ]:
while(True):
    print("---------------------------------")
    print('1.Trim the video\n''2.crop the video\n''3.Resize_Frame\n''4.Exit\n')
    opt = int(input('enter the option'))
    if(opt == 1):
      video = input('enter the path of the video')
      while(True):
        print("enter the timestamp as a manner like.... 'minute|second|milisecond\n'please give the all three values\n",'otherwise you can put 0')
        time_stamp_first = input('enter start_time:')
        time_stamp_second = input('enter end_time:')
        print('processing.....')
        
        path = input('please give the file path where you want to store the new video')
        res_1 = getFrame(video, time_stamp_first, time_stamp_second,path)
        if(res_1 == -1):
          print('please enter again from the begining')
          continue
        else:
          audi_file =trimAudio(video, time_stamp_first, time_stamp_second, path)
          print('finished........')
          print('video saved.. and audio save.....')
          print('now merging audio and video......')
          audioMergevideo(res_1, audi_file,path)
          print('finished......')
          break
    elif(opt == 2):
      video = input('enter the path of the video')
      while(True):
        x1= int(input('enter the x1 coordinate:'))
        y1 =int(input('enter the y1 coordinate:'))
        x2 = int(input('enter the x2 coordinate:'))
        y2 = int(input('enter the y2 coordinate:'))
        print('processing....')
        path = input('please give the file path where you want to store the new video')
        res = getCropFrame(video, x1, y1, x2, y2, path)
        if (res ==-1):
          print("please enter again from the begining")
          continue
        else:
          print(res)
          audio_file = getAudio(video,path)
          print('video and audio saved.....')
          print('now merging...')
          audioMergevideo(res, audio_file,path)
          print('finished.....')
          break
    elif(opt == 3):
      video = input('enter the path of the video')
      print('1.with out maintaining the aspect ratio\n''2.maintaining the aspect ration\n')
      press = int(input('enter the option'))
      while True:
        if press == 1:
          w_vid = int(input('enter the weight'))
          h_vid = int(input('enter the height'))
          print('processing....')
          path = input('please give the file path where you want to store the new video')
          res_2 = getFrameResize(video, w_vid, h_vid, path)
          if res_2 == -1:
            print('please enter again from the begining')
            continue
          else:
            audio_file = getAudio(video,path)
            print('video and audio saved....')
            audioMergevideo(res_2, audio_file,path)
            print("finished...")
            break
        if press == 2:
          w_vid = int(input('enter the weidth'))
          print('processing....')
          path = input('please give the file path where you want to store the new video')
          res_3 = getFrameAspect(video, w_vid, path)
          if res_3 == -1:
            print('please enter again from the begining')
            continue
          else:
            audio_file = getAudio(video,path)
            print('video and audio saved....')
            audioMergevideo(res_3, audio_file,path)
            print("finished...")
            break

    elif(opt == 4):
      print('Thank you for using this application')
      break

